## *Task-Trained Network*
<img src="https://www.biobanking.com/wp-content/uploads/2020/01/brain_connectivity_copy-39p-696x392.jpg" height="160" />

***
***

# **Before every experiment**

## Restart the kernel
- Select `Kernel` > `Restart Kernel and Clear All Outputs`

## Reset notebook
Discard any changes made so we can pull from remote next:

- Select `Git` > `Reset to remote`

- `Close all windows...` should be checked (discard changes, not save them)

- Click `Proceed` (this notebook will need to be re-opened)

## Pull notebook changes 

- `Git` > `Pull from remote` (with force, if necessary)

## Check environment
-  JupyterLab should display `venv: .venv` in top right of window

## Update environment & reset logs
- Run this cell and lookout for errors

In [ ]:
%%cmd
pdm update np_workflows --no-self
rmdir /s /q logs

***
***
# **Without mouse on stage**

In [ ]:
import np_logging
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.task_trained_network as TTN

np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select session
Re-run cell this cell if mouse ID is changed

In [ ]:
selected_session = TTN.stim_session_select_widget(mouse)

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
experiment: np_workflows.WithLims = TTN.new_experiment(mouse, user, selected_session.session)

session, platform_json = experiment.session, experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab = isinstance(experiment, TTN.Hab)

***
## Checks before starting

### **Ephys day 2?** 
Don't forget to adjust probe targeting!

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
[critical]

In [ ]:
experiment.initialize_and_test_services()

***
## Dip probes

In [ ]:
if not hab:
    np_workflows.di_widget(session)

In [ ]:
np_workflows.probe_coords_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_experiment_surface_image')

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget()

***
## When cartridge is lowered

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
np_workflows.photodoc_widget('brain_surface_image')

***
## Probe insertion

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()
    np_workflows.isi_widget(mouse.lims)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer

In [ ]:
if not hab:
    np_workflows.print_countdown_timer(minutes=.1 if experiment.ttn_session.value == 'pretest' else 10)

In [ ]:
np_workflows.probe_notes_widget(session)

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_insertion_surface_image')

In [ ]:
np_workflows.pre_stim_check_widget()

***
## Start devices recording

In [ ]:
experiment.start_recording()

***
## Start stimulus scripts
mapping | main | opto

In [ ]:
experiment.run_stim_scripts()

***
## Stop recording

In [ ]:
experiment.stop_recording_after_stim_finished()

***
## Before removing probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
if not hab:
    np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_files()